In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [ ]:
#data
#TODO

In [3]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        #init your encoder
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden
    
    def init_hidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [ ]:
class DecoderWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, encoder_hidden_size, attention_size):
        super(DecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_size = attention_size
        self.wa = nn.Linear(attention_size, hidden_size)
        self.ua = nn.Linear(attention_size, hidden_size * 2)
        self.va = nn.Linear(1, hidden_size)
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.ia = nn.Linear(encoder_hidden_size + hidden_size, hidden_size)
        
    def forward(self, input, hidden, s_pre, encoder_outputs):
        e_i = torch.zeros(1, len(encoder_outputs))
        for j in range(len(encoder_outputs)):
            e_i[j] = a(s_pre, encoder_outputs[j])
        alpha_i = F.softmax(e_i)
        c_i = torch.bmm(alpha_i.unsqueeze(0), ncoder_outputs.unsqueeze(0))
        c_i = torch.cat((c_i, s_pre), 1)
        s_i = self.ia(c_i)
        
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, s_i)
        return output, hidden
            
    def a(s_pre, h_j):
        return torch.mm(self.va, F.tanh(self.wa(s_pre) + self.ua(h_j)))